In [ ]:
import os
import numpy as np
import matplotlib
import torch
!pip install mat73
import mat73
import matplotlib.pyplot as plt
import csv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split,TensorDataset
from torchvision import transforms, utils
import time
import pandas as pd
import scipy.io
import sklearn.metrics
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from torchvision.transforms import ToTensor

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
np.random.seed(1)

from scipy import signal

from sklearn.metrics import confusion_matrix

import seaborn as sn
import pandas as pd

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import LeaveOneGroupOut






In [ ]:
## if with EMG
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split
#file_name_train=r"/content/drive/My Drive/Colab Notebooks/RFMG/data/Data_allfilt_0.5_7.mat"

## EMG signal included in dataset 

ExpVer='emg1'
FiltName='filt'+'_0.1_5'
file_name_train=r"/content/drive/My Drive/Colab Notebooks/RFMG/data/"+ExpVer+"/Data_17_6_comb"+FiltName+".mat"

#file_name_train=r"/content/drive/My Drive/Colab Notebooks/RFMG/data/"+ExpVer+"/Data_17_6_combfilt_0.05_10.mat"
data = mat73.loadmat(file_name_train)


feature_all = data['feature_all'][:,:,:]    ## waveform * channel * case 
feature_all=np.swapaxes(feature_all,0,2)
feature_all_c = data['feature_all_complex'][:,:,:]    ## waveform * channel * case 
feature_all_c=np.swapaxes(feature_all_c,0,2)
feature_all_emg = data['feature_all_emg'][:,:,:]    ## waveform * channel * case 
feature_all_emg=np.swapaxes(feature_all_emg,0,2)
self_channel=[0,1,10,11,20,21,30,31]

feature_all2=np.hstack((feature_all,feature_all_c))
label_all = data['label_all']
caseNum_all = data['caseNum_all']

Chan_Name = data['Chan_Name']

#truncate time length 
fs=500
ind=np.arange(fs*1, fs*6, 1, dtype=int)
feature_all2=feature_all2[:,:,ind]
feature_all_emg=feature_all_emg[:,:,ind]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split
#file_name_train=r"/content/drive/My Drive/Colab Notebooks/RFMG/data/Data_allfilt_0.5_7.mat"

ExpVer='2_24'
FiltName='filt'+'_0.1_5'
file_name_train=r"/content/drive/My Drive/Colab Notebooks/RFMG/data/"+ExpVer+"/Data_17_6_comb"+FiltName+".mat"

#file_name_train=r"/content/drive/My Drive/Colab Notebooks/RFMG/data/"+ExpVer+"/Data_17_6_combfilt_0.05_10.mat"
data = mat73.loadmat(file_name_train)


feature_all = data['feature_all'][:,:,:]    ## waveform * channel * case 
feature_all=np.swapaxes(feature_all,0,2)
feature_all_c = data['feature_all_complex'][:,:,:]    ## waveform * channel * case 
feature_all_c=np.swapaxes(feature_all_c,0,2)
self_channel=[0,1,10,11,20,21,30,31]

feature_all2=np.hstack((feature_all,feature_all_c))
label_all = data['label_all']
caseNum_all = data['caseNum_all']

Chan_Name = data['Chan_Name']

#truncate time length 
fs=500
ind=np.arange(fs*1, fs*6, 1, dtype=int)
feature_all2=feature_all2[:,:,ind]
feature_all=feature_all[:,:,ind]

Mounted at /content/drive


In [ ]:
feature_all=feature_all2
## EMG signal included in dataset 
#feature_all=np.hstack((feature_all2,feature_all_emg))

# time  range [0.3 4.7]
## frequency range [0,15.71]
def stft_2d(X):

  
  fs = 500
  f_th=56
  Zxx_all=np.zeros([X.shape[0],X.shape[1],f_th,f_th])
  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
      phase_data=X[i,j,:] 
      f, t, Zxx = signal.stft(phase_data, fs, nperseg=fs*0.6,noverlap = fs*0.52,nfft=fs*3.5,boundary = None)
      Zxx_out=Zxx[0:f_th,:]
      Zxx_all[i,j,:,:]=abs(Zxx_out)
  Zxx_all=np.array(Zxx_all)  
  return Zxx_all

feature_2d_stft=stft_2d(feature_all)

# t (0.5,4.5)  f (0,6.7)
def stft_2d_v2(X):

  
  fs = 500
  f_th=41
  Zxx_all=np.zeros([X.shape[0],X.shape[1],f_th,f_th])
  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
      phase_data=X[i,j,:] 
      f, t, Zxx = signal.stft(phase_data, fs, nperseg=fs*1,noverlap = fs*0.9,nfft=fs*6,boundary = None)
    
      Zxx_out=Zxx[0:f_th,:]
      Zxx_all[i,j,:,:]=abs(Zxx_out)
  Zxx_all=np.array(Zxx_all)  
  return Zxx_all

feature_2d_stft2=stft_2d_v2(feature_all)

size_f=30
f=np.linspace(0.1,10,size_f)
def WT(dataIn):
  Ds=20
  fs=500/Ds
  dataIn = signal.resample(dataIn, int(dataIn.shape[2]/Ds),axis=2)
  # freqeucny range 
  widths =np.round(fs/f)  # frequency to widths range 

  Zxx_all=np.zeros([dataIn.shape[0],dataIn.shape[1],dataIn.shape[2],size_f])
  for i in range(dataIn.shape[0]):
    for j in range(dataIn.shape[1]):
      phase_data=dataIn[i,j,:]
     
      cwtmatr = signal.cwt(phase_data, signal.ricker, widths) # CWT transform 
      Zxx_out=cwtmatr.T 
      Zxx_all[i,j,:,:]=abs(Zxx_out)
  Zxx_all=np.array(Zxx_all)  
  return Zxx_all
    
feature_2d_cwt=WT(feature_all)

def WT2(dataIn):
  Ds=20
  fs=500/Ds
  dataIn = signal.resample(dataIn, int(dataIn.shape[2]/Ds),axis=2)
  # freqeucny range 
  widths =np.round(fs/f)  # frequency to widths range 

  Zxx_all=np.zeros([dataIn.shape[0],dataIn.shape[1],dataIn.shape[2],size_f])
  for i in range(dataIn.shape[0]):
    for j in range(dataIn.shape[1]):
      phase_data=dataIn[i,j,:]
     
      cwtmatr = signal.cwt(phase_data, signal.morlet2, widths) # CWT transform 
      Zxx_out=cwtmatr.T 
      Zxx_all[i,j,:,:]=abs(Zxx_out)
  Zxx_all=np.array(Zxx_all)  
  return Zxx_all
    
feature_2d_cwt2=WT2(feature_all)

import pywt
size_f=30
f=np.linspace(0.1,10,size_f)
def WT3(dataIn):
  Ds=20
  fs=500/Ds
  dataIn = signal.resample(dataIn, int(dataIn.shape[2]/Ds),axis=2)
  # freqeucny range 
  widths =np.round(fs/f)  # frequency to widths range 

  Zxx_all=np.zeros([dataIn.shape[0],dataIn.shape[1],dataIn.shape[2],size_f])
  for i in range(dataIn.shape[0]):
    for j in range(dataIn.shape[1]):
      phase_data=dataIn[i,j,:]
      cwtmatr, freqs = pywt.cwt(phase_data, widths,"cgau1")
      #print(cwtmatr.shape) # CWT transform 
      Zxx_out=cwtmatr.T 
      Zxx_all[i,j,:,:]=abs(Zxx_out)
  Zxx_all=np.array(Zxx_all)  
  return Zxx_all
feature_2d_cwt3=WT3(feature_all)  

feature_all=feature_all.astype('float16')
feature_2d_stft=feature_2d_stft.astype('float16')
feature_2d_stft2=feature_2d_stft2.astype('float16')[:,:,:40,:40]
feature_2d_cwt=feature_2d_cwt.astype('float16')
feature_2d_cwt2=feature_2d_cwt2.astype('float16')
feature_2d_cwt3=feature_2d_cwt3.astype('float16')

np.savez_compressed("/content/drive/My Drive/Colab Notebooks/RFMG/data/"+ExpVer+"/data_all_multi6_"+FiltName+".npz",feature_all=feature_all,\
                    feature_2d_stft=feature_2d_stft,feature_2d_stft2=feature_2d_stft2,\
                    feature_2d_cwt=feature_2d_cwt,\
                    feature_2d_cwt2=feature_2d_cwt2,feature_2d_cwt3=feature_2d_cwt3,\
                    label_all=label_all,caseNum_all=caseNum_all)
# np.savez_compressed("/content/drive/My Drive/Colab Notebooks/RFMG/data/"+ExpVer+"/data_all_multi6_wEMG"+FiltName+".npz",feature_all=feature_all,\
#                     feature_2d_stft=feature_2d_stft,feature_2d_stft2=feature_2d_stft2,\
#                     feature_2d_cwt=feature_2d_cwt,\
#                     feature_2d_cwt2=feature_2d_cwt2,feature_2d_cwt3=feature_2d_cwt3,\
#                     label_all=label_all,caseNum_all=caseNum_all)

/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1815: UserWarning: Input data is complex, switching to return_onesided=False
  warnings.warn('Input data is complex, switching to '
/usr/local/lib/python3.7/dist-packages/scipy/signal/wavelets.py:472: ComplexWarning: Casting complex values to real discards the imaginary part
  output[ind] = convolve(data, wavelet_data, mode='same')
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:103: ComplexWarning: Casting complex values to real discards the imaginary part


In [ ]:
feature_all=feature_all.astype('float16')
feature_2d_stft=feature_2d_stft.astype('float16')
feature_2d_stft2=feature_2d_stft2.astype('float16')[:,:,:40,:40]
feature_2d_cwt=feature_2d_cwt.astype('float16')
feature_2d_cwt2=feature_2d_cwt2.astype('float16')
feature_2d_cwt3=feature_2d_cwt3.astype('float16')

np.savez_compressed("/content/drive/My Drive/Colab Notebooks/RFMG/data/"+ExpVer+"/data_all_multi6_"+FiltName+".npz",feature_all=feature_all,\
                    feature_2d_stft=feature_2d_stft,feature_2d_stft2=feature_2d_stft2,\
                    feature_2d_cwt=feature_2d_cwt,\
                    feature_2d_cwt2=feature_2d_cwt2,feature_2d_cwt3=feature_2d_cwt3,\
                    label_all=label_all,caseNum_all=caseNum_all)

In [ ]:
## delete bas cases in one participant's dataset 

from google.colab import drive
drive.mount('/content/drive')
ExpVer='2_19'
file_name_data=r"/content/drive/My Drive/Colab Notebooks/RFMG/data/"+ExpVer+"/data_all_multi6_wEMGfilt_0.1_5.npz"
data=np.load(file_name_data)
feature_2d_stft=data['feature_2d_stft']
feature_2d_stft2=data['feature_2d_stft2']
feature_2d_cwt=data['feature_2d_cwt']
feature_2d_cwt2=data['feature_2d_cwt2']
feature_2d_cwt3=data['feature_2d_cwt3']
feature_all=data['feature_all']
label_all=data['label_all']
caseNum_all=data['caseNum_all']


del_per=np.where(caseNum_all==7)[0]

feature_2d_cwt=np.delete(feature_2d_cwt,del_per,axis=0)
feature_2d_cwt2=np.delete(feature_2d_cwt2,del_per,axis=0)
feature_2d_cwt3=np.delete(feature_2d_cwt3,del_per,axis=0)
feature_2d_stft=np.delete(feature_2d_stft,del_per,axis=0)
ffeature_2d_stft2=np.delete(feature_2d_stft2,del_per,axis=0)
feature_all=np.delete(feature_all,del_per,axis=0)

label_all=np.delete(label_all,del_per,axis=0)
caseNum_all=np.delete(caseNum_all,del_per,axis=0)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
FiltName='filt'+'_0.1_5'
np.savez_compressed("/content/drive/My Drive/Colab Notebooks/RFMG/data/"+ExpVer+"/data_all_multi6_wEMG"+FiltName+".npz",feature_all=feature_all,\
                    feature_2d_stft=feature_2d_stft,feature_2d_stft2=feature_2d_stft2,\
                    feature_2d_cwt=feature_2d_cwt,\
                    feature_2d_cwt2=feature_2d_cwt2,feature_2d_cwt3=feature_2d_cwt3,\
                    label_all=label_all,caseNum_all=caseNum_all)